# Using a Vector Database to Recommend Movies

SOLUTION: https://youtu.be/dCkn6WTadgY

Vector search is certainly critical for generative AI, but also has lots of other interesting applications as well. One very common one is building personalized recommendations. In this exercise, we'll take a small diversion and build a quick movie recommender using a vector database.

For this exercise we'll use the [MovieLens Latest Small Dataset](https://grouplens.org/datasets/movielens/latest/), which contains 100,000 ratings and 3,600 tags applied to 9,000 movies by 600 users. The strategy we'll use is to create embeddings for the movies based on the user ratings. Then if a user rated a particular movie highly, we'll recommend "similar" movies, as determined by the embeddings

In [1]:
import lancedb

import numpy as np
import pandas as pd

/home/tumaaa/work/repos/udacity_nano_degree_generative_ai/.venv/lib/python3.10/site-packages/tqdm/auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


The dataset is included along with this exercise:

In [2]:
!ls ml-latest-small

links.csv  movies.csv  ratings.csv  README.txt	tags.csv


## Loading data

Let's start by reading in the `ratings.csv` file. We'll use this to compute the content embeddings

In [3]:
ratings = pd.read_csv('./ml-latest-small/ratings.csv', header=0)
ratings

,userId,movieId,rating,timestamp
0,1,1,4.0,964982703
1,1,3,4.0,964981247
2,1,6,4.0,964982224
3,1,47,5.0,964983815
4,1,50,5.0,964982931
...,...,...,...,...
100831,610,166534,4.0,1493848402
100832,610,168248,5.0,1493850091
100833,610,168250,5.0,1494273047
100834,610,168252,5.0,1493846352


## Computing ratings

Use the ratings dataframe from above and create a new reviews dataframe of users (index) and movies (columns). Each entry (i, j) in the dataframe will be the rating that user_i gave to movie_j. If no such pair exists, then fill in the value 0.

**HINT** In Excel this would be called a **pivot** table

In [4]:
reviewmatrix = ratings.pivot(index="userId", columns="movieId", values="rating").fillna(0)

In [5]:
reviewmatrix.head()


movieId,1,2,3,4,5,6,7,8,9,10,...,193565,193567,193571,193573,193579,193581,193583,193585,193587,193609
userId,,,,,,,,,,,,,,,,,,,,,
1,4.0,0.0,4.0,0.0,0.0,4.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
2,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
3,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
4,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
5,4.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0


## Computing embeddings

Now let's use [matrix factorization](https://www.cs.cmu.edu/~mgormley/courses/10601-s17/slides/lecture25-mf.pdf) to extract content embeddings.

Please compute the content embeddings from the reviewmatrix dataframe and name the result `embeddings`.

**HINT**
1. SVD is a popular matrix factorization technique
2. If you're not sure which of the SVD results to use as the content embeddings, look at the shape of the results

In [6]:
matrix = reviewmatrix.values
_, _, vh = np.linalg.svd(matrix)
embeddings = vh.T

In [7]:
print(f"{embeddings.shape=}")


embeddings.shape=(9724, 9724)


## Metadata

Read in the `movies.csv` and `links.csv` files and make sure it is aligned with the embeddings dataframe.

**HINT** pandas provides `reindex` functionality to help with data alignment

In [8]:
movies = pd.read_csv('./ml-latest-small/movies.csv', header=0)
movies = movies.set_index("movieId").reindex(reviewmatrix.columns)
movies

,title,genres
movieId,,
1,Toy Story (1995),Adventure|Animation|Children|Comedy|Fantasy
2,Jumanji (1995),Adventure|Children|Fantasy
3,Grumpier Old Men (1995),Comedy|Romance
4,Waiting to Exhale (1995),Comedy|Drama|Romance
5,Father of the Bride Part II (1995),Comedy
...,...,...
193581,Black Butler: Book of the Atlantic (2017),Action|Animation|Comedy|Fantasy
193583,No Game No Life: Zero (2017),Animation|Comedy|Fantasy
193585,Flint (2017),Drama


In [9]:
# now do this for links
links = pd.read_csv('./ml-latest-small/links.csv', header=0)
links.set_index("movieId").reindex(reviewmatrix.columns)

,imdbId,tmdbId
movieId,,
1,114709,862.0
2,113497,8844.0
3,113228,15602.0
4,114885,31357.0
5,113041,11862.0
...,...,...
193581,5476944,432131.0
193583,5914996,445030.0
193585,6397426,479308.0


## Create vector database table

Let's create a table with the following fields:

1. an integer movie id field
2. a vector field of embeddings
3. a string field of genres
4. a string field for the movie title
5. an integer field for the imdb_id

First, we'll create a pydantic model named `Content` for these fields. For the vector field, use the `lancedb.pydantic.vector` as a shorthand for the field type. Note that you'll need to pass in the number of dimensions.

In [10]:
from lancedb.pydantic import vector, LanceModel

class Content(LanceModel):
    movie_id: int
    vector: vector(embeddings.shape[1])
    genres: str
    title: str
    imdb_id: int
        
    @property
    def imdb_url(self) -> str:
        return f"https://www.imdb.com/title/tt{self.imdb_id}"

Let's prepare a list of python dicts with all of the data

In [11]:
values = list(zip(*[reviewmatrix.columns,
                    embeddings, 
                    movies["genres"], 
                    movies["title"], 
                    links["imdbId"], 
                    links["tmdbId"]]))
keys = Content.__annotations__.keys()
data = [dict(zip(keys, v)) for v in values]

data[0]

{'movie_id': 1,
 'vector': array([-0.0704499 ,  0.02759119, -0.07844388, ..., -0.00101062,
        -0.00101062,  0.00389649]),
 'genres': 'Adventure|Animation|Children|Comedy|Fantasy',
 'title': 'Toy Story (1995)',
 'imdb_id': 114709}

Now please connect to the local database at ~/.lancedb
and create the LanceDB table named "movielens_small".

**HINT** you've seen this in a previous exercise

In [13]:
import pyarrow as pa
table_name = "movielens_small"
data = pa.Table.from_pylist(data, schema=Content.to_arrow_schema())

db = lancedb.connect("file:///~/.lancedb")

In [16]:
db.drop_table(table_name)
table = db.create_table(table_name, data=data)

## Generating recommendations

Finally we're ready to generate recommendations based on content vector similarity.

For this exercise please fill in the rest of the function to generate recommendations

**HINT** It's easier if you use the pydantic integration to convert results

In [17]:
def get_recommendations(title: str) -> list[(int, str, str)]:
    # First we retrieve the vector for the input title
    query_vector = (table.to_lance()
                    .to_table(filter=f"title='{title}'")["vector"].to_numpy()[0])
    # Please write the code to search for the 5 most similar titles
    results=table.search(query_vector).limit(5).to_pydantic(Content)
    # For each result, return the movie_id, title, and imdb_url
    return [(c.movie_id, c.title, c.imdb_url) for c in results]

If a user watched the movie titled "Moana (2016)", what should we recommend to the user?

In [18]:
get_recommendations("Moana (2016)")

[(166461, 'Moana (2016)', 'https://www.imdb.com/title/tt6023444'),
 (2008,
  'This World, Then the Fireworks (1997)',
  'https://www.imdb.com/title/tt119910'),
 (5256, 'Stolen Summer (2002)', 'https://www.imdb.com/title/tt76729'),
 (1427, 'Turbulence (1997)', 'https://www.imdb.com/title/tt120550'),
 (941, 'Mark of Zorro, The (1940)', 'https://www.imdb.com/title/tt32762')]

What about "Rogue One: A Star Wars Story (2016)"?

In [ ]:
get_recommendations("Rogue One: A Star Wars Story (2016)")

Do these look reasonable? How would you improve this recommender system?